# Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)


Mounted at /content/drive/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install unrar

In [ ]:
!unrar x "/content/drive/MyDrive/train.rar" "./data"

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Extracting  ./data/train/Ji Chang Wook/JiChangWook (48).jpg               40%  OK 
Extracting  ./data/train/Ji Chang Wook/JiChangWook (49).jpg               40%  OK 
Extracting  ./data/train/Ji Chang Wook/JiChangWook (5).jpg                40%  OK 
Extracting  ./data/train/Ji Chang Wook/JiChangWook (50).jpg               40%  OK 
Extracting  ./data/train/Ji Chang Wook/JiChangWook (51).jpg               40%  OK 
Extracting  ./data/train/Ji Chang Wook/JiChangWook (6).jpg                40%  OK 
Extracting  ./data/train/Ji Chang Wook/JiChangWook (7).jpg                40%  OK 
Extracting  ./data/train/Ji Chang Wook/JiChangWook (8).jpg                40%  OK 
Extracting  ./data/train/Ji Chang Wook/JiChangWook (9).jpg                40%  OK 
Creating    ./data/train/Joe Biden                                    OK
Extracting  ./data/train/Joe Biden/JoeBide

Số lượng floder = class


In [ ]:
import os
print("floder-class:", len(os.listdir('./data/train')))

floder-class: 150


In [ ]:
!pip install pytube --upgrade
!pip install fastapi
!pip install kaleido
!pip install python-multipart
!pip install uvicorn
!pip install pyparsing==2.4.7
!pip install numpy==1.23.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
bigframes 0.13.0 requires fsspec>=2023.3.0, but you have fsspec 2022.11.0 which is incompatible.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatibl

#Build clip image embeddings

In [ ]:
!pip install clip-retrieval autofaiss

In [ ]:
!clip-retrieval inference --input_dataset ./data/train/ --output_folder ./data/embedding_folder

The number of samples has been estimated to be 6988
Starting the worker
dataset is 13
Starting work on task 0
warming up with batch size 256 on cpu
done warming up in 206.97615361213684s
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
 sample_per_sec 4 ; sample_count 6988 

In [ ]:
!ls ./data/embedding_folder

img_emb  metadata  stats


In [ ]:
from pathlib import Path
import pandas as pd
data_dir = Path("./data/embedding_folder/metadata")
df = pd.concat(
    pd.read_parquet(parquet_file)
    for parquet_file in data_dir.glob('*.parquet')
)
image_list = df["image_path"].tolist()

In [ ]:
image_list

['data/train/Amber Heard/AmberHeard (1).jpg',
 'data/train/Amber Heard/AmberHeard (10).jpg',
 'data/train/Amber Heard/AmberHeard (11).jpg',
 'data/train/Amber Heard/AmberHeard (12).jpg',
 'data/train/Amber Heard/AmberHeard (13).jpg',
 'data/train/Amber Heard/AmberHeard (14).jpg',
 'data/train/Amber Heard/AmberHeard (15).jpg',
 'data/train/Amber Heard/AmberHeard (16).jpg',
 'data/train/Amber Heard/AmberHeard (17).jpg',
 'data/train/Amber Heard/AmberHeard (18).jpg',
 'data/train/Amber Heard/AmberHeard (19).jpg',
 'data/train/Amber Heard/AmberHeard (2).jpg',
 'data/train/Amber Heard/AmberHeard (20).jpg',
 'data/train/Amber Heard/AmberHeard (21).jpg',
 'data/train/Amber Heard/AmberHeard (22).jpg',
 'data/train/Amber Heard/AmberHeard (23).jpg',
 'data/train/Amber Heard/AmberHeard (24).jpg',
 'data/train/Amber Heard/AmberHeard (25).jpg',
 'data/train/Amber Heard/AmberHeard (26).jpg',
 'data/train/Amber Heard/AmberHeard (27).jpg',
 'data/train/Amber Heard/AmberHeard (28).jpg',
 'data/train/Am

# Build a KNN index

In [ ]:

!autofaiss build_index --embeddings="./data/embedding_folder/img_emb" \
                    --index_path="./data/knn.index" \
                    --index_infos_path="./data/infos.json" \
                    --metric_type="ip" \
                    --max_index_query_time_ms=10 \
                    --max_index_memory_usage="4GB"

2023-11-16 14:53:29,824 [INFO]: Using 2 omp threads (processes), consider increasing --nb_cores if you have more
2023-11-16 14:53:29,825 [INFO]: Launching the whole pipeline 11/16/2023, 14:53:29
2023-11-16 14:53:29,825 [INFO]: Reading total number of vectors and dimension 11/16/2023, 14:53:29
100% 1/1 [00:00<00:00, 9597.95it/s]
2023-11-16 14:53:29,869 [INFO]: There are 6988 embeddings of dim 512
2023-11-16 14:53:29,869 [INFO]: >>> Finished "Reading total number of vectors and dimension" in 0.0443 secs
2023-11-16 14:53:29,870 [INFO]: 	Compute estimated construction time of the index 11/16/2023, 14:53:29
2023-11-16 14:53:29,870 [INFO]: 		-> Train: 16.7 minutes
2023-11-16 14:53:29,870 [INFO]: 		-> Add: 0.0 seconds
2023-11-16 14:53:29,870 [INFO]: 		Total: 16.7 minutes
2023-11-16 14:53:29,870 [INFO]: 	>>> Finished "Compute estimated construction time of the index" in 0.0002 secs
2023-11-16 14:53:29,870 [INFO]: 	Checking that your have enough memory available to create the index 11/16/2023, 